In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))


In [50]:
import os
import sys
import json
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import AssetConfigTxn, wait_for_confirmation
from algosdk import constants
import base64
from account import *

In [31]:
address, private_key, passphrase = generate_algorand_keypair()

My address: H3V5SSOS7QIEM6FJVMP4HHL4AKZE6WG2ZAQLHUGGJFSEI3ZOF5D5OSVZHI
My private key: eKDVIPXv5m+doMFBndm7zf8bg9o/TJF3hQukwPVKSL8+69lJ0vwQRnipqx/DnXwCsk9Y2sggs9DGSWREby4vRw==
My passphrase: audit final motor young soccer fossil agent script popular grid target wolf dawn patch yard equip vendor beyond alarm apart twist bargain where absent decade


In [32]:
key_data = {
    'address': address,
    'private_key': private_key,
    'passphrase': passphrase
}

with open('../data/key_pair.json', 'w') as f:
    json.dump(key_data, f)

In [48]:

def first_transaction_example(private_key, my_address):
    algod_address = "http://localhost:4001"
    algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
    algod_client = algod.AlgodClient(algod_token, algod_address)
    accoun_info = algod_client.account_info(my_address)
    print(f'account info: {accoun_info}')

    params = algod_client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    # params.flat_fee = True
    # params.fee = constants.MIN_TXN_FEE 
    receiver = "HZ57J3K46JIJXILONBBZOHX6BKPXEM2VVXNRFSUED6DKFD5ZD24PMJ3MVA"
    note = "My first transaction".encode()
    amount = 1000000
    unsigned_txn = transaction.PaymentTxn(my_address, params, receiver, amount, None, note)
    signed_txn = unsigned_txn.sign(private_key)
    #submit transaction
    txid = algod_client.send_transaction(signed_txn)
    print("Successfully sent transaction with txID: {}".format(txid))

    # wait for confirmation 
    try:
        confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 4)  
    except Exception as err:
        print(err)
        return

    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
    print("Amount transfered: {} microAlgos".format(amount) )    
    print("Fee: {} microAlgos".format(params.fee) ) 


    account_info = algod_client.account_info(my_address)
    print("Final Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

In [49]:
first_transaction_example(private_key, address)

account info: {'address': 'H3V5SSOS7QIEM6FJVMP4HHL4AKZE6WG2ZAQLHUGGJFSEI3ZOF5D5OSVZHI', 'amount': 3999000, 'amount-without-pending-rewards': 3999000, 'apps-local-state': [], 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0}, 'assets': [], 'created-apps': [], 'created-assets': [], 'min-balance': 100000, 'pending-rewards': 0, 'reward-base': 27521, 'rewards': 0, 'round': 22237102, 'status': 'Offline', 'total-apps-opted-in': 0, 'total-assets-opted-in': 0, 'total-created-apps': 0, 'total-created-assets': 0}
Successfully sent transaction with txID: AJCI6CGXNKGJ5XXHL2RC7RUEPQBCS4HRDIHRQUELT2MPJO5HR7FQ
Transaction information: {
    "confirmed-round": 22237104,
    "pool-error": "",
    "txn": {
        "sig": "4gEYwHVrPhKc47dtPytpccl8msmxwRvIoMsjexCB9imJKRwfILiPw/pR4ODWTHY5oLOIj7vHRkPW0C8vUwB4Dg==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "fv": 22237102,
            "gen": "testnet-v1.0",
            "gh": "SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/

In [53]:
# create an assset

# CREATE ASSET
# Get network params for transactions before every transaction.
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction
txn = AssetConfigTxn(
    sender=address,
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="10 Academy",
    asset_name="10ACAD",
    manager=address,
    reserve=address,
    freeze=address,
    clawback=address,
    url="www.10academy.org", 
    decimals=0)
# Sign with secret key of creator
stxn = txn.sign(private_key)
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))


transaction {_struct:{} Sig:[124 87 56 204 54 140 175 175 245 149 241 31 61 65 87 197 230 134 128 143 63 243 151 92 41 240 92 213 3 182 161 241 191 177 251 28 227 178 165 168 167 64 210 219 52 39 216 248 17 202 116 181 217 189 181 36 143 241 15 185 116 32 130 4] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[] Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[]} Txn:{_struct:{} Type:acfg Header:{_struct:{} Sender:H3V5SSOS7QIEM6FJVMP4HHL4AKZE6WG2ZAQLHUGGJFSEI3ZOF5D5OSVZHI Fee:{Raw:1000} FirstValid:22237315 LastValid:22238315 Note:[] GenesisID:testnet-v1.0 GenesisHash:JBR3KGFEWPEE5SAQ6IWU6EEBZMHXD4CZU6WCBXWGF57XBZIJHIRA Group:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA Lease:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] RekeyTo:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ} Keyreg